In [15]:
import pymssql  
conn = pymssql.connect(
    # server='demosqlwesteurope.database.windows.net',
    # user='student_01',
    # password='!MiptSql_01',

    server='mipt.database.windows.net',
    user='student03',
    password='Student_03',

    database='demo',
)

In [16]:
from tabulate import tabulate

def exec(query: str):
    cursor = conn.cursor()  
    cursor.execute(query) 
    
def q(query: str, *args):
    cursor = conn.cursor()  
    cursor.execute(query) 
    row = cursor.fetchone()  
    rows = []
    while row:  
        rows.append(row)
        # for i, element in enumerate(row):
            # print(str(row[i]), end="\t\t")
        # print()
        row = cursor.fetchone()  
    print(tabulate(rows, headers=args))

In [9]:
q("""--sql
    SELECT name, database_id, create_date  
    FROM sys.databases;  
""")

------  -  --------------------------
master  1  2021-11-08 06:18:56.827000
demo    5  2021-11-08 15:01:07.873000
------  -  --------------------------


In [17]:
q("""--sql
SELECT database_id, is_read_only, collation_name, compatibility_level  
FROM sys.databases WHERE name = 'demo';  
""")

q("SELECT @@version")

q("""--sql
    select schema_name(t.schema_id) as schema_name,
        t.name as table_name,
        t.modify_date
    from sys.tables t
    order by schema_name,
        table_name;
""")

-  -----  ----------------------------  ---
5  False  Latin1_General_100_BIN2_UTF8  150
-  -----  ----------------------------  ---
----------------------------------------
Microsoft SQL Azure (RTM) - 12.0.2000.8
	Sep 18 2021 19:01:34
	Copyright (C) 2019 Microsoft Corporation
----------------------------------------
-----------  --------------  --------------------------
distributor  attributesItem  2021-11-08 15:02:59.840000
distributor  branch          2021-11-08 15:02:59.963000
distributor  company         2021-11-08 15:03:00.083000
distributor  currency        2021-11-08 15:03:00.200000
distributor  ddp             2021-11-08 15:03:00.327000
distributor  item            2021-11-08 15:03:00.447000
distributor  remains         2021-11-08 15:03:00.573000
distributor  sales           2021-11-08 15:03:00.700000
distributor  salesManager    2021-11-08 15:03:00.837000
distributor  singleSales     2021-11-08 15:03:00.957000
distributor  specification   2021-11-08 15:03:01.073000
----------

In [12]:
q("SELECT top(10) * from distributor.sales")

----  ---------  -  ---  ----  ----------  --  ---------
2369  327476000  4  100  7322  2014-08-14  -1   -3275.9
2369  337471000  4  100  7322  2014-08-14  -1   -2347.18
2040  16303      4  198  7315  2014-09-18  -6   -2305.38
2040  18656      4  198  7315  2014-09-18  -8  -14578.1
1461  2506       4       5507  2014-08-26  -4   -6575.08
4067  401037     4  100  7325  2014-09-10  -1   -2667.5
1240  5A3125C00  1  201  2376  2014-02-28  -1   -1583.67
5103  110099     1  201  7165  2014-08-22  -1   -2520
3569  357514000  1  108  3007  2014-01-10  -1   -4727.7
2973  444500     1  108  2947  2014-01-10  -1   -1773.51
----  ---------  -  ---  ----  ----------  --  ---------


**1 -**

**2**

**3**

4

5

6

7

8\. no

9\. no

10\. no

11\. no

12\. no

13\. no

14\. no

15\. no

16\. no

17\. -

18. Сколько в среднем обслуживает клиентов менеджер филиала.

19. Сколько всего клиентов обслужил филиал за определенный период.

In [18]:
q("""--sql
    select count(sales.checkId), branch.branchName
    from distributor.sales
    inner join distributor.branch on (sales.branchId = branch.branchId)
    group by sales.branchId, branch.branchName
""", "Clients served", "Branch")

  Clients served  Branch
----------------  ---------------
          360500  Екатеринбург
            1063  ИнтернетМагазин
          773092  Москва
          424876  Новосибирск
            6124  Пятигорск
          351504  Самара


20. Какой менеджер обслужил в филиале, максимальное кол-во клиентов

In [19]:
q("""--sql
    with temp(managerId, checksSold) as (
        select salesManagerId, count(sales.checkId)
        from distributor.sales
        where salesManagerId is not Null
        group by salesManagerId
    )

    select managerId, surname, [names], checksSold
    from temp
    inner join distributor.salesManager on (salesManager.salesManagerId = temp.managerId)
    where checksSold = (
        select max(checksSold)
        from temp
    )
""", "id", "surname", "name", "clients")

21. Какой менеджер, принес максимальную выручку в филиале за определенный месяц

In [57]:
q("""--sql
    select max(salesRub) as maxSalesRub, salesManagerId
    from distributor.sales 
    where
        branchId = 4 and
        month(dateId) = 8
    group by salesManagerId
""", "sales in rub", "Manager ID")

  sales in rub    Manager ID
--------------  ------------
      185100
       39019.8            53
      218825              87
      195663              88
      343230              95
      228000              96
      122248             100
      140315             131
      202600             141
       36000             152
       40826.5           165
      146192             197
      138942             198
       65312.4           204
       36127.6           234
       75093.4           238
      411211             240


22. Рассчитать средний чек клиенту по выбранному менеджеру

In [59]:
q("""--sql
    select top(10) avg(salesRub), salesManager.surname
    from distributor.sales
    inner join distributor.salesManager on (salesManager.salesManagerId = sales.salesManagerId)
    group by sales.salesManagerId, salesManager.surname
""", "Average check", "Sales manager surname")

  Average check  Sales manager surname
---------------  -----------------------
        1926.75  Аброскина
        1094.02  Андреева
        3219.91  Антонов
       20129.1   Бакуров
       19984.7   Баранов
        2612.61  Баранов
        4378.14  Березина
        7175.51  БОТВИЧ
        9753.49  Валов
        3729.43  Васильев


23. Рассчитать средний чек клиента по филиалу

In [60]:
q("""--sql
    select avg(salesRub), branchName
    from distributor.sales
    inner join distributor.branch on (branch.branchId = sales.branchId)
    group by branch.branchId, branchName
""", "Avarage check in rubles", "Branch")

  Avarage check in rubles  Branch
-------------------------  ---------------
                  4584.42  Екатеринбург
                  2142.03  ИнтернетМагазин
                  7309.87  Москва
                  5978.9   Новосибирск
                 14346.4   Пятигорск
                  4976.34  Самара


24. Средний чек клиента по менеджерам внутри филиалов

In [10]:
# TODO 

25

In [11]:
# TODO 

26

In [12]:
# TODO 

27. Рассчитать АВС товарных позиций

In [13]:
# TODO 

28\. -

29\. -

30\. -

31\. -

32. Есть менеджеры, которые продают в более чем одном филиале, найти их!

In [63]:
%%time

q("""--sql
    with sms(managerId, numberOfBranches)
    as
    (
        select sm.salesManagerId, count(distinct branch.branchId)
        from distributor.sales
        inner join distributor.branch on (sales.branchId = branch.branchId)
        inner join distributor.salesManager as sm on (sales.salesManagerId = sm.salesManagerId)
        group by sm.salesManagerId
        having count(distinct branch.branchId) > 1
    )

    select managerId, numberOfBranches, salesManager.surname, salesManager.[names]
    from sms
    inner join distributor.salesManager on (sms.managerId = salesManager.salesManagerId)
""", "ID", "Number of branches", "Surname", "Name")

  ID    Number of branches  Surname      Name
----  --------------------  -----------  ---------------
   7                     2  Березина     Стелла
   9                     2  Валов        Игорь
  10                     2  Васильев     Дмитрий
  13                     2  Воробьева    Юлия
  19                     2  Долганов     Геннадий
  20                     2  Дудкина      Татьяна
  28                     2  Исакова      Евгения
  31                     2  Карпухина    Елена Андреевна
  34                     2  Ковалевская  Алёна
  36                     2  Комаров      Эдуард
  37                     2  Комарова     Василина
  47                     2  Мальшин      Олег
  53                     2  Меньшиков    Сергей
  54                     2  Минеева      Ксения
  67                     3  Рензяк       Диана
  74                     2  Семенов      Олег
  76                     2  Собакин      Денис
  78                     3  Ставицкая    Анастасия
  83                    

33. Найти сумму продаж по менеджерам, со следующими условиями. Какими условиями?

34. Выгрузки с датами, различными. The fuck?

35. Показать, что среднее кол-во чеков и среднее по транзакции это разные вещи.

In [15]:
# todo

36\. Найти количество уникальных счетов при условиях. Каких условиях?